<a href="https://colab.research.google.com/github/DionKimmm/2019_Summer_DL_Prof_Seok/blob/master/190731_RNN_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 4글자의 영단어를 이용해 3글자의 x데이터로 4번째 글자인 y데이터를 완성시키는 문제

# 준비물
* 원핫 인코딩 사용을 위해 연관 배열을 만든다.
* {'a': 0, 'b': 1, 'c': 2, ..., 'z':25}


In [0]:
import tensorflow as tf
import numpy as np

In [0]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [3]:
print(num_dic)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [4]:
print(dic_len)

26


In [0]:
# wor -> X, d -> Y
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [0]:
def make_batch(seq_data):
  input_batch = []
  target_batch = []
    
  for seq in seq_data:
    # 여기서 생성하는 input_batch 와 target_batch 는 알파벳 배열의 인덱스 번호
    # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
    input = [num_dic[n] for n in seq[:-1]]
    # 3, 3, 15, 4, 3 ...
    target = num_dic[seq[-1]]
    # one-hot 인코딩
    # if input is [0, 1, 2]:
    # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.],  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.], [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
    input_batch.append(np.eye(dic_len)[input])
    target_batch.append(target)

  return input_batch, target_batch


In [0]:
# 입력을 다이나믹하게 하려면??

# 학습, 테스트, 레이블 데이터 불러오기

In [0]:
# 옵션 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3         # 타입 스텝: [1 2 3] => 3, RNN 을 구성하는 시퀀스의 갯수

# 입력값 크기. 알파벳에 대한 one-hot 인코딩이므로 26개, 출력값도 동일하게 26개
n_input = n_class = dic_len


# 신경망 모델

 비용함수에 sparse_softmax_cross_entropy_with_logits 을 사용하므로
출력값과의 계산을 위한 원본값의 형태는 one-hot vector가 아니라 인덱스 숫자를 그대로 사용하기 때문에 다음처럼 하나의 값만 있는 1차원 배열을 입력값으로 받는다. 
* [3] [3] [15] [4] ... <br>
기존처럼 one-hot 인코딩을 사용한다면 입력값의 형태는 [None, n_class] 이여야 한다. 
* W와 b를 사용하지 않고 아래와 같이 tf.layers.dense 함수를 사용해도 됨.
* model = tf.layers.dense(outputs, n_class)


In [9]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# RNN 셀을 생성합니다.
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5) # Dropout 기법
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

# tf.nn.dynamic_rnn 함수를 이용해 순환 신경망을 만듭니다.
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# 최종 결과는 one-hot 인코딩 형식으로 만듭니다
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

W0731 04:33:01.240673 140318481389440 deprecation.py:323] From <ipython-input-9-013f09025ae6>:8: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0731 04:33:01.246411 140318481389440 deprecation.py:323] From <ipython-input-9-013f09025ae6>:11: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0731 04:33:01.248425 140318481389440 deprecation.py:323] From <ipython-input-9-013f09025ae6>:14: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0731

# 손실함수 및 최적화

In [0]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


# 훈련 데이터를 신경망 모델 학습

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')


Epoch: 0001 cost = 3.008123
Epoch: 0002 cost = 2.049457
Epoch: 0003 cost = 1.559072
Epoch: 0004 cost = 1.039742
Epoch: 0005 cost = 1.021140
Epoch: 0006 cost = 0.465791
Epoch: 0007 cost = 0.449311
Epoch: 0008 cost = 0.475116
Epoch: 0009 cost = 0.302419
Epoch: 0010 cost = 0.199285
Epoch: 0011 cost = 0.467535
Epoch: 0012 cost = 0.347861
Epoch: 0013 cost = 0.134216
Epoch: 0014 cost = 0.120158
Epoch: 0015 cost = 0.208308
Epoch: 0016 cost = 0.260094
Epoch: 0017 cost = 0.163456
Epoch: 0018 cost = 0.206605
Epoch: 0019 cost = 0.037248
Epoch: 0020 cost = 0.244063
Epoch: 0021 cost = 0.282579
Epoch: 0022 cost = 0.052751
Epoch: 0023 cost = 0.079461
Epoch: 0024 cost = 0.161344
Epoch: 0025 cost = 0.089042
Epoch: 0026 cost = 0.066235
Epoch: 0027 cost = 0.009713
Epoch: 0028 cost = 0.013642
Epoch: 0029 cost = 0.149451
Epoch: 0030 cost = 0.008266
최적화 완료!


# 테스트 데이터를 이용한 최종 식별 결과 확인

In [12]:
# 레이블값이 정수이므로 예측값도 정수로 변경해줍니다.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)

# one-hot 인코딩이 아니므로 입력값을 그대로 비교합니다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val) 


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
